In [18]:
import boto3
from IPython.display import FileLink
%load_ext dotenv
%dotenv

sagemaker_client = boto3.client('sagemaker')

response = sagemaker_client.list_training_jobs(
    SortBy='CreationTime',
    SortOrder='Descending',
    MaxResults=1
)

training_jobs = response.get('TrainingJobSummaries', [])

if not training_jobs:
    raise ValueError("No training jobs found.")
else:
    latest_training_job_name = training_jobs[0]['TrainingJobName']
    print(f"Latest Training Job Name: {latest_training_job_name}")

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
Latest Training Job Name: xgb-clf-training-container-iwtx02hfntro-yfGP1iQ1cG

2024-10-20 17:51:10 Starting - Preparing the instances for training
2024-10-20 17:51:10 Downloading - Downloading the training image
2024-10-20 17:51:10 Training - Training image download completed. Training in progress.
2024-10-20 17:51:10 Uploading - Uploading generated training model
2024-10-20 17:51:10 Completed - Training job completed


In [16]:
import boto3
import re

s3_client = boto3.client('s3')

training_job_description = sagemaker_client.describe_training_job(
    TrainingJobName=latest_training_job_name
)

debug_hook_config = training_job_description.get('DebugHookConfig', {})
s3_output_path = debug_hook_config.get('S3OutputPath', None)

if not s3_output_path:
    raise ValueError("No S3OutputPath found in DebugHookConfig.")

match = re.match(r's3://([^/]+)/?(.*)', s3_output_path)
if match:
    bucket_name, prefix_base = match.groups()
    if not prefix_base:
        prefix_base = ''
else:
    raise ValueError(f"Invalid S3 URI: {s3_output_path}")

prefix = f"{prefix_base}{latest_training_job_name}/rule-output/CreateXgboostReport/"

response = s3_client.list_objects_v2(
    Bucket=bucket_name,
    Prefix=prefix
)

if 'Contents' in response:
    for obj in response['Contents']:
        key = obj['Key']
        if 'xgboost_report' in key:
            file_name = key.split('/')[-1]
            s3_client.download_file(bucket_name, key, file_name)
            print(f"Downloaded {file_name}")
else:
    print("No XGBoost report files found in the specified S3 path.")

Downloaded xgboost_report.html
Downloaded xgboost_report.ipynb


In [17]:
# display("Click link below to view the profiler report", FileLink("reports/profiler/profiler-output/profiler-report.html"))
display("Click link below to view the XGBoost Training report", FileLink("xgboost_report.html"))

'Click link below to view the XGBoost Training report'

C:\Users\kamil\Documents\football_project\aws_pipline_clone\aws_football_predictor\program\code\debuggeroutput\report\xgboost_report.html